In [12]:
import cx_Oracle
import pandas as pd
from pandas import ExcelWriter

In [13]:
v_batch_id     = 180912213000

filename_input = 'account.xlsx'
filename_output = 'account-gsi_partynum.xlsx'

In [15]:
# CRMAT connection
user_name = "username"
pwd = "password"
hostname = "localhost"
port = "1553"
service_name = "xe"
conn_str = "{0}/{1}@{2}:{3}/{4}".format(user_name, pwd, hostname, port, service_name)

In [17]:
# open connection
conn = cx_Oracle.connect(conn_str)

# create cursor
cursor = cx_Oracle.Cursor(conn)

In [18]:
reader = pd.ExcelFile(filename_input)
writer = ExcelWriter(filename_output)

for sheet in reader.sheet_names:
	accounts = reader.parse(sheet)
	account_regid = accounts['Registry Id']
	v_process_type = 'Enrich-%s' % sheet
	
	rows = []
	for x in account_regid.values:
		rows.append( ( '%s' % x, v_batch_id, v_process_type ) )

	# populate this sheet
	insert_stmt = """
		insert into MIS_STALE_ACCOUNTS (party_number,batch_id,process_type) 
			values (:1, :2, :3)
		"""
		
	cursor.executemany(insert_stmt, rows)
	conn.commit()
	
	# run below query in SQL Developer:
	update_stmt = """
		update mis_stale_accounts bg 
			set (party_id,GSI_PARTY_NUM) = (
				select OPF.party_id, OPF.extn_attribute_char001
				from HZ_ORGANIZATION_PROFILES opf
				where opf.party_number = bg.party_number
			) where gsi_party_num is null and batch_id=%d and process_type='%s'
		""" % (v_batch_id, v_process_type)
		
	cursor.execute(update_stmt)
	conn.commit()
	
	select_stmt = """
		select bg.party_number,bg.gsi_party_num from mis_stale_accounts bg 
			where batch_id=%d and process_type='%s'
	""" % (v_batch_id, v_process_type)
	
	cursor.execute(select_stmt)
	rowset = cursor.fetchall() 
	
	gsi_num = pd.DataFrame(columns=['Registry Id', 'GSI PartyNum'], data=rowset)
	
	accounts2 = pd.merge(accounts, gsi_num, on='Registry Id')
	
	accounts2.to_excel(writer, sheet_name=sheet, header=True, index=False)


writer.save()

conn.commit()
cursor.close()
conn.close()